In [2]:
from source.single_question.questions_loader import QuestionLoader
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\USER\anaconda3\envs\questionnaire-dashboard\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
questions = QuestionLoader().load_questions()

def get_q_text(q):
    if q.choices is not None:
        text = f"Question: {q.question_text}\nMultiple Choices:\n"        
        for k, v in q.choices.items():
            text += f"Choice {k} = {v}\n"
        return text
    elif q.ancestor is not None:
        return f"Question: {q.ancestor}\n Answer: {q.question_text}"
    else:
        return q.question_text
        
text = [get_q_text(q) for q in questions.questions]
q_objs = [q for q in questions.questions]

In [4]:

# Step 1: Embed all descriptions
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2") # Fast & good for short texts
desc_embeddings = model.encode(text, convert_to_tensor=True)


C:\Users\USER\anaconda3\envs\questionnaire-dashboard\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [5]:

# Step 2: Define search function
def search_questionnaires(query, desc_embeddings, model, top_k=5):
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarities = cosine_similarity([query_embedding], desc_embeddings)[0]
    top_indices = similarities.argsort()[-top_k:][::-1]
    # return similarities, df.iloc[top_indices][['Abbreviated Name', desc_col]]
    return top_indices


In [15]:
query = "sexual trauma"
top_k = 10
exclusion_questionnaires = ["suicide_form_clin", "opening", "mini_kid_sum_stu", "mini_kid_sum_clin"]

def exclude(exclusion_questionnaires, q_objs, desc_embeddings):
    clean_q_objs, clean_desc_embeddings = [], []
    for q, emb in zip(q_objs, desc_embeddings):
        if q.questionnaire_name not in exclusion_questionnaires:
            clean_q_objs.append(q)
            clean_desc_embeddings.append(emb)

    return clean_q_objs, clean_desc_embeddings

In [16]:
clean_q_objs, clean_desc_embeddings = exclude(exclusion_questionnaires, q_objs, desc_embeddings)

In [17]:
top_indices = search_questionnaires(query, clean_desc_embeddings, model, top_k)
print(top_indices)

[ 658 1025  635 1002  991  624  370 1547 1551 1549]


C:\Users\USER\anaconda3\envs\questionnaire-dashboard\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [18]:
q_texts = []
for i in range(top_k):
    q = clean_q_objs[top_indices[i]]
    if q.question_text not in q_texts:
        print(f"top {i}")
        print(f"Text: \n{q.question_text}")
        print(f"Questionnaire:\t{q.questionnaire_name}")
        if q.choices is not None:
            print(f"Choices:\n{q.choices}\n")
        print(f"Variable Name:\t{q.variable_name}\n\n")
        q_texts.append(q.question_text)



top 0
Text: 
סיבת הטיפול
Questionnaire:	opening_ps_m
Variable Name:	ps_demograph_11_1_m


top 2
Text: 
תקריות אחרות כלפי אחרים (אנא תארו)
Questionnaire:	moas_m
Variable Name:	moas_10_m


top 4
Text: 
נא פרט/י מה זה היה
Questionnaire:	cts_f
Variable Name:	cts_p_4_1_f


top 7
Text: 
Details
Questionnaire:	screening_form
Variable Name:	screen_35_1


